In [8]:
import pandas as pd
import mysql.connector

csv_files = [
    ("KTCL_bus_info.csv", "Goa"),
    ("WB_bus_info.csv", "West Bengal"),
    ("NB_bus_info.csv", "Nagaland"),
    ("JK_bus_info.csv", "Jammu and Kashmir"),
    ("KAAC_bus_info.csv", "Assam"),
    ("PUNJAB_bus_info.csv", "Punjab"),
    ("CTU_bus_info.csv", "Chandigarh"),
    ("ASTC_bus_info.csv", "Assam"),
    ("HRTC_bus_info.csv", "Himachal Pradesh"),
    ("SBSTC_bus_info.csv", "West Bengal")
]

# Reading and adding CSV's to DataFrame
df_list = []
for file, state in csv_files:
    df = pd.read_csv(file)
    df['State'] = state 
    df_list.append(df)

# Concatenate all DataFrames
result_df = pd.concat(df_list, ignore_index=True)

# Data cleaning
result_df['Price'] = result_df['Price'].str.replace('INR ', '', regex=False)
result_df = result_df.dropna()
result_df.insert(0, 'id', range(1, len(result_df) + 1))

# Save cleaned data to a CSV
result_df.to_csv("bus_routes_cleaned.csv", index=False)

# Database connection
my_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='3008',
    auth_plugin='mysql_native_password'
)

my_cursor = my_connection.cursor()

# Creating the database 
my_cursor.execute("CREATE DATABASE IF NOT EXISTS redbus")

# Switch to new database
my_cursor.execute("USE redbus")

# Creating the table 
my_cursor.execute("""
CREATE TABLE IF NOT EXISTS bus_routes (
    id INT PRIMARY KEY,
    Route_Name TEXT,
    Route_Link TEXT,                            
    Bus_Name TEXT,
    Bus_Type TEXT,
    Departing_Time TEXT,
    Duration TEXT,
    Reaching_Time TEXT,
    Star_Rating TEXT,
    Price TEXT,
    Seat_Availability TEXT,
    State TEXT 
)
""")
expected_columns = ['id', 'Route_Name', 'Route_Link', 'Bus_Name', 'Bus_Type', 'Departing_Time', 'Duration', 'Reaching_Time', 'Star_Rating', 'Price', 'Seat_Availability', 'State']
result_df = result_df[expected_columns]

# Inserting cleaned data into the database
for index, row in result_df.iterrows():
    try:
        print(f"Inserting row {index}: {tuple(row)}")  
        my_cursor.execute("""
        INSERT INTO bus_routes (id, Route_Name, Route_Link, Bus_Name, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Availability, State) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, tuple(row))
    except mysql.connector.IntegrityError as err:
        if "1062" in str(err):  # Duplicate entry error
            print(f"Duplicate entry for row {index}, skipping.")
        else:
            print(f"Error inserting row {index}: {err}")

# Commit and close the connection
my_connection.commit()
my_cursor.close()
my_connection.close()


Inserting row 0: (1, 'Pune to Goa', 'https://www.redbus.in/bus-tickets/pune-to-goa', 'IntrCity SmartBus', 'Bharat Benz A/C Seater /Sleeper (2+1)', '21:00', '11h 05m', '08:05', 4.4, '609', '18 Seats available', 'Goa')
Duplicate entry for row 0, skipping.
Inserting row 1: (2, 'Pune to Goa', 'https://www.redbus.in/bus-tickets/pune-to-goa', 'Ashray Amrutyog', 'A/C Sleeper (2+1)', '22:00', '11h 30m', '09:30', 4.0, '900', '18 Seats available', 'Goa')
Duplicate entry for row 1, skipping.
Inserting row 2: (3, 'Pune to Goa', 'https://www.redbus.in/bus-tickets/pune-to-goa', 'Ashray Travels', 'Non A/C Seater / Sleeper (2+1)', '21:15', '10h 30m', '07:45', 4.2, '500', '15 Seats available', 'Goa')
Duplicate entry for row 2, skipping.
Inserting row 3: (4, 'Pune to Goa', 'https://www.redbus.in/bus-tickets/pune-to-goa', 'Ashray Vip Yellows', 'A/C Sleeper (2+1)', '20:30', '11h 30m', '08:00', 4.1, '900', '15 Seats available', 'Goa')
Duplicate entry for row 3, skipping.
Inserting row 4: (5, 'Pune to Goa',